In [1]:
import requests
import json
import time
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
collections = []

for n in range(0,6000,500):
    print('Started collections offset: ',n)

    offset = str(n)
    url = "http://api-mainnet.magiceden.dev/v2/collections?offset="+offset+"&limit=500"
    payload = {}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    response_collections_json = response.json()

    for i in range(0,len(response_collections_json)):
        symbol = response_collections_json[i]['symbol']

        print('Started inner collection (collections/inner offset): ',n,'/',i)
        print('Symbol: ',symbol)
        
        url = "http://api-mainnet.magiceden.dev/v2/collections/"+symbol+"/stats"
        payload = {}
        headers = {}
        response = requests.request("GET", url, headers=headers, data=payload)
        response_symbol_json = response.json()
        print('response_symbol_json complete')

        url = "http://api-mainnet.magiceden.dev/v2/collections/"+symbol+"/listings?offset=0&limit=1"
        payload = {}
        headers = {}
        response_listing = requests.request("GET", url, headers=headers, data=payload)
        response_listing_json = response_listing.json()
        print('response_listing_json complete')
        
        if len(response_listing_json) > 0:
            response_symbol_json['mint'] = response_listing_json[0]['tokenMint']
        else:
            response_symbol_json['mint'] = '0'

        collections += [response_symbol_json]

        print('Ended inner collection: ',n,'/',i)
        print('Symbol: ',symbol)
        # print(collections)
        time.sleep(0.001)
    
    print('Ended collections offset: ',n)

In [ ]:
with open('magiceden_stats.json','w') as outfile:
    json.dump(collections, outfile)

In [ ]:
# Part 2 starting clean

collections = []

for n in range(3052,6052,500):
    print('STARTED collections offset: ',n)

    offset = str(n)
    url = "http://api-mainnet.magiceden.dev/v2/collections?offset="+offset+"&limit=500"
    payload = {}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    response_collections_json = response.json()

    for i in range(0,len(response_collections_json)):
        symbol = response_collections_json[i]['symbol']

        print('STARTED inner collection (collections/inner offset): ',n,'/',i)
        print('Symbol: ',symbol)
        
        url = "http://api-mainnet.magiceden.dev/v2/collections/"+symbol+"/stats"
        payload = {}
        headers = {}
        response = requests.request("GET", url, headers=headers, data=payload)
        response_symbol_json = response.json()
        print('response_symbol_json complete')

        url = "http://api-mainnet.magiceden.dev/v2/collections/"+symbol+"/listings?offset=0&limit=1"
        payload = {}
        headers = {}
        response_listing = requests.request("GET", url, headers=headers, data=payload)
        response_listing_json = response_listing.json()
        print('response_listing_json complete')
        
        if len(response_listing_json) > 0:
            response_symbol_json['mint'] = response_listing_json[0]['tokenMint']
        else:
            response_symbol_json['mint'] = '0'

        collections += [response_symbol_json]

        print('ENDED inner collection: ',n,'/',i)
        print('Symbol: ',symbol)
        # print(collections)
        time.sleep(0.001)
    
    print('ENDED collections offset: ',n)

In [ ]:
with open('magiceden_stats_2.json','w') as outfile:
    json.dump(collections, outfile)

In [26]:
df1 = pd.read_json('magiceden_stats.json')
df2 = pd.read_json('magiceden_stats_2.json')

magiceden_df = pd.concat([df1,df2], ignore_index=True)
magiceden_df.drop(['listedCount','avgPrice24hr'], axis=1, inplace=True)
magiceden_df.drop_duplicates(subset=['symbol','mint'], keep='last', inplace=True, ignore_index=True)
magiceden_df.dropna(axis=0, subset=['floorPrice'], inplace=True)
magiceden_df.fillna(value={'volumeAll':'0'}, inplace=True)
magiceden_df.rename(columns={
    'floorPrice': 'floor_price',
    'volumeAll': 'trading_volume'
}, inplace=True)

magiceden_df['floor_price'] = magiceden_df.floor_price.apply(
    lambda x: float(x) / 10**9
)
magiceden_df['trading_volume'] = magiceden_df.trading_volume.apply(
    lambda x: float(x) / 10**9
)
magiceden_df = magiceden_df[magiceden_df['mint'] != '0']
magiceden_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5675 entries, 0 to 5823
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   symbol          5675 non-null   object 
 1   floor_price     5675 non-null   float64
 2   trading_volume  5675 non-null   float64
 3   mint            5675 non-null   object 
dtypes: float64(2), object(2)
memory usage: 221.7+ KB


In [27]:
magiceden_df.head()

,symbol,floor_price,trading_volume,mint
0,sol_chocked_man,0.400,1.200000,CR2rcWaHWJgcMSzdewSs7S6QLryJZnt5eMQHQf9mQULS
1,_peguimonkey,0.380,27.740000,ByX4fiQNFjeHiL9xCXaee39XhWeptC5zWxdwvHCcqC8b
2,soa,1.700,0.000000,GTVeFkLD5bjfY1imyEkkhcSTocKE1Vnkxc4gC2T8JUbs
3,asciipunk,0.027,15.334369,H68LSpMfzHsAvwUistfeGXhCN3xfqpyMrq2BodEYuXrJ
4,koala_solana,0.100,1.389000,DeRtQA4dxGJPP3w83vw5VsB7NZjLxBGTXgWSsKK6FXnj


In [28]:
magiceden_df.to_json('magiceden_cleaned.json')
magiceden_df[['symbol','floor_price','mint']].to_json('magiceden_cleaned_floor.json')

In [ ]:
mints = [mint for mint in magiceden_df.mint]
# for mint in magiceden_df.mint:
#     mints.append(mint)

print(mints)

In [38]:
for i in range(0, len(mints), 1135):
    start = i 
    end = i + 1135
    print(i)
    with open('mints'+str(i)+'.txt', 'w') as txt:
        txt.write("'" + "','".join(mints[start:end]) + "'")

# print(len(mints))
# with open('mints.txt', 'w') as txt:
#     txt.write("'" + "','".join(mints) + "'")

0
1135
2270
3405
4540
